In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
import tqdm
import plotly.express as px
import plotly.graph_objects as go
import datetime

c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\pdoub\Desktop\python_projects\junk-drawer\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\pdoub\Desktop\python_projects\junk-draw

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_parquet("reddit_legal_cluster_test_results.parquet")
df.reset_index(inplace=True, drop=True)
df.sort_index(inplace=True)

In [4]:
df.shape

(5000, 14)

In [5]:
df.tail(2)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,State,kmeans_label,topic_title
4998,4994,1475721018,https://www.reddit.com/r/legaladvice/comments/...,5639t2,I will preface this by admitting I'm an idiot ...,"(CO) First Ticket ever, Careless Driving",driving,4,"[0.004459863429254688, 0.003479491143272819, 0...",209,"""Legal options for a teenager facing a Careles...",NJ,10,Legal Topics in Traffic Violations
4999,8627,1589322350,https://www.reddit.com/r/legaladvice/comments/...,gilww3,Hey folks;\n\nI'm currently designing a video ...,[OH] Advice on safe contracts for freelancers/...,contract,1,"[0.010599094232140971, -0.01940659953784975, -...",191,"""Protecting Intellectual Property and Ensuring...",OR,6,Compilation of Legal Topics


In [20]:
from sklearn.base import BaseEstimator, TransformerMixin
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

class TopicEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, text_col, threshold=0.1):
        self.text_col = text_col
        self.threshold = threshold
        self.vectorizer_model = CountVectorizer(stop_words = 'english')
        self.topic_model = BERTopic(
            vectorizer_model = self.vectorizer_model,
            nr_topics = 'auto',
            calculate_probabilities=True)

    def fit(self, X, y=None):
        docs = list(X[self.text_col])
        self.topic_model.fit(docs)
        return self

    def transform(self, X):
        docs = list(X[self.text_col])
        # X = X.join(self.topic_model.get_document_info(docs))
        topic_distr, _ = self.topic_model.approximate_distribution(docs, window = 4, calculate_tokens=True)
        X['multiple_topics'] = list(map(
            lambda doc_topic_distr: list(map(
                lambda y: y[0], filter(lambda x: x[1] >= self.threshold, 
                                    (enumerate(doc_topic_distr)))
            )), topic_distr
        ))
        X = self.one_hot_encode_topics(X, 'multiple_topics')
        return X

    def one_hot_encode_topics(self, df, column_name):
        df[column_name] = df[column_name].astype('string')
        one_hot_list = []
        for idx, row in df.iterrows():
            topics = eval(row[column_name])
            one_hot_row = {}
            for topic in topics:
                one_hot_row[f'topic_{topic}'] = 1
            one_hot_list.append(one_hot_row)
        one_hot_df = pd.DataFrame(one_hot_list)
        one_hot_df.fillna(0, inplace=True)
        one_hot_df = one_hot_df.astype(int)

        # Sort the columns
        one_hot_df = one_hot_df.sort_index(axis=1)

        df.reset_index(drop=True, inplace=True)
        df = pd.concat([df, one_hot_df], axis=1)
        return df

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from fastembed.embedding import FlagEmbedding as Embedding
import umap
import numpy as np

class EmbeddingUmapTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, text_column, model_name="BAAI/bge-base-en-v1.5", max_length=512, n_neighbors=10, metric='cosine', min_dist=0.0, n_components=5):
        self.text_column = text_column
        self.model_name = model_name
        self.max_length = max_length
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.min_dist = min_dist
        self.n_components = n_components
        self.umap_model = umap.UMAP(angular_rp_forest=True, n_neighbors=n_neighbors, metric=metric, min_dist=min_dist, n_components=n_components)

    def fit(self, X, y=None):
        self.embedding_model = Embedding(model_name=self.model_name, max_length=self.max_length)
        documents = ("passage: " + text for text in X[self.text_column])
        embeddings = self.embedding_model.embed(documents)
        self.umap_model.fit(np.vstack(embeddings))
        return self

    def transform(self, X):
        self.embedding_model = Embedding(model_name=self.model_name, max_length=self.max_length)
        documents = ("passage: " + text for text in X[self.text_column])
        embeddings = self.embedding_model.embed(documents)
        reduced_embeddings = self.umap_model.transform(np.vstack(embeddings))
        for i in range(reduced_embeddings.shape[1]):
            X[f'umap_{i}'] = reduced_embeddings[:, i]
        return X

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from keyphrase_vectorizers import KeyphraseCountVectorizer
import spacy
from keybert import KeyBERT
from collections import Counter

class KeywordVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, max_features=None, text_col_name=None):
        self.max_features = max_features
        self.text_col_name = text_col_name
        self.nlp = spacy.load("en_core_web_lg")
        self.kw_model = KeyBERT(model=self.nlp)
        self.vectorizer = KeyphraseCountVectorizer(
            spacy_pipeline=self.nlp,
            )
        self.vectorizer_model = None
        self.vocabulary = None

    def fit(self, X, y=None):
        docs = X[self.text_col_name]
        keywords = self.kw_model.extract_keywords(
            docs,
            vectorizer=self.vectorizer,
            top_n=10,
            use_mmr=True)
        keyword_counts = Counter(k[0] for keyword in keywords for k in keyword)
        self.vocabulary = [k for k, _ in keyword_counts.most_common(self.max_features)]
        self.vectorizer_model = TfidfVectorizer(vocabulary=self.vocabulary)
        self.vectorizer_model.fit(docs)
        return self

    def transform(self, X):
        df = X.copy()
        vectors = self.vectorizer_model.transform(df[self.text_col_name])
        new_columns = pd.DataFrame(vectors.toarray(), columns=['count_vec_' + str(i) for i in range(vectors.shape[1])])
        df.reset_index(drop=True, inplace=True)
        df = pd.concat([df, new_columns], axis=1)
        return df


In [24]:
import spacy
import textacy
import coreferee
from typing import Counter
from gensim.models import FastText
import spacy.cli

def check_and_install_spacy_models(models):
    for model in models:
        try:
            # Try to load the model
            spacy.load(model)
        except OSError:
            # If model is not installed, download and install it
            print(f"{model} not found. Downloading and installing...")
            spacy.cli.download(model)
# List of models to check and install
models = ["en_core_web_md", "en_core_web_lg"]

check_and_install_spacy_models(models)

class TripleExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, text_column, risk_keywords, n_top_triples=20, corpus=None):
        self.nlp = spacy.load("en_core_web_lg")
        self.nlp1 = spacy.load('en_core_web_lg')
        self.nlp1.add_pipe('coreferee')
        self.nlp1 = spacy.load("en_core_web_lg")
        if "coreferee" not in self.nlp1.pipe_names:
            self.nlp1.add_pipe('coreferee')
        self.nlp2 = spacy.load("en_core_web_md")
        self.text_column = text_column
        self.risk_keywords = risk_keywords
        self.n_top_triples = n_top_triples
        self.corpus = corpus
        self.top_triples = None
        self.risk_triples = None
        
        # If a corpus is provided, train Word2Vec on it
        if corpus:
            sentences = [text.split() for text in corpus]
            self.fasttext_model = FastText(sentences, window=5, min_count=1, workers=4)    
    
    def fit(self, X, y=None):
        # Extract all triplets from the training data
        all_triples = self._extract_all_triples(X[self.text_column])
        
        # Find the most common triplets
        triple_counter = Counter(all_triples)
        self.top_triples = [triple for triple, count in triple_counter.most_common(self.n_top_triples)]
        
        # Find the risk-related triplets
        self.risk_triples = [triple for triple in all_triples if any(self.fasttext_model.wv.similarity(keyword, word) > 0.8 for word in triple for keyword in self.risk_keywords)]
        
        return self
    
    def transform(self, X):
        # Initialize a dictionary to store the counts of each risk keyword
        risk_keyword_counts = {keyword: [] for keyword in self.risk_keywords}
        top_feature_vectors = []
        for texts in tqdm.tqdm(X[self.text_column]):
            text_triples = self._extract_triples_from_text(texts)
            top_features = [int(triple in text_triples) for triple in self.top_triples]
            top_feature_vectors.append(top_features)
            # Count the occurrences of each risk keyword in the text triples
            for keyword in self.risk_keywords:
                risk_keyword_counts[keyword].append(sum(keyword in triple for triple in text_triples))
        # Convert the dictionary to a DataFrame
        risk_keyword_df = pd.DataFrame(risk_keyword_counts)
        
        # Convert the lists of feature vectors to DataFrames
        top_feature_df = pd.DataFrame(top_feature_vectors, columns=["top_triples_" + str(triple) for triple in self.top_triples])
        
        # Clean the column names in the top_feature_df
        top_feature_df.columns = top_feature_df.columns.str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '').str.replace("('", "").str.replace("',_'", "_").str.replace("')", "")
        
        # Concatenate the new features to the original DataFrame
        X_extended = pd.concat([X.reset_index(drop=True), top_feature_df, risk_keyword_df], axis=1)
        
        return X_extended

    def _extract_all_triples(self, X):
        # This method extracts all triplets from a list of texts
        all_triples = []
        for texts in X:
            text_triples = self._extract_triples_from_text(texts)
            all_triples.extend(text_triples)
        return all_triples

    def _extract_triples_from_text(self, text):
        triples = []

        # Coreference resolution
        text = self.coref_resolve(text)

        # Split compound sentences into simple sentences
        simple_sentences = self.compound_to_simple(text)

        for sentence in simple_sentences:
            doc = self.nlp(sentence)
            svo_triples = list(textacy.extract.subject_verb_object_triples(doc))
            for triple in svo_triples:
                subj = "_".join(map(str, triple.subject))
                obj = "_".join(map(str, triple.object))
                verb = "_".join(map(str, triple.verb))
                triples.append((subj, verb, obj))
        return triples

    def coref_resolve(self, text):
        doc1 = self.nlp1(text)
        tok_list = list(token.text for token in doc1)
        c = 0
        for chain in doc1._.coref_chains:
            for mention in chain:
                res1 = [doc1._.coref_chains.resolve(doc1[i]) for i in mention]
                res = [r for r in res1 if r is not None]
                if len(res) != 0:
                    if len(res[0]) == 1:
                        tok_list[mention[0] + c] = str(res[0][0])
                    elif len(res[0]) > 1:
                        tok_list[mention[0] + c] = str(res[0][0])
                        for j in range(1, len(res[0])):
                            tok_list.insert(mention[0] + c + j, str(res[0][j]))
                            c = c + 1
        textres = " ".join(tok_list)
        return textres

    def compound_to_simple(self, sentence):
        doc = self.nlp2(sentence)
        root_token = None
        for token in doc:
            if (token.dep_ == "ROOT"):
                root_token = token
        other_verbs = []
        for token in doc:
            ancestors = list(token.ancestors)
            if (token.pos_ == "VERB" and len(ancestors) < 3 and token != root_token):
                other_verbs.append(token)
        token_spans = []
        all_verbs = [root_token] + other_verbs
        for other_verb in all_verbs:
            first_token_index = len(doc)
            last_token_index = 0
            this_verb_children = list(other_verb.children)
            for child in this_verb_children:
                if (child not in all_verbs):
                    if (child.i < first_token_index):
                        first_token_index = child.i
                    if (child.i > last_token_index):
                        last_token_index = child.i
            token_spans.append((first_token_index, last_token_index))
        sentence_clauses = []
        for token_span in token_spans:
            start = token_span[0]
            end = token_span[1]
            if (start < end):
                clause = doc[start:end]
                sentence_clauses.append(clause)
        sentence_clauses = sorted(sentence_clauses, key=lambda tup: tup[0])
        clauses_text = [clause.text for clause in sentence_clauses]
        return clauses_text


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

X_train, X_test = train_test_split(df, test_size=0.4, random_state=42)

In [25]:
risk_keywords = ["liability", "insurance", "should_do"]

corpus_ = df['body'].tolist()
corpus = [d for d in corpus_]

pipeline = Pipeline([
    ('topic_encoder', TopicEncoder(text_col="body")),
    ('kw_count_vectorizer', KeywordVectorizer(max_features=50, text_col_name="body")),
    ('triplet_extractor', TripleExtractor(
        'body', 
        risk_keywords=risk_keywords, 
        n_top_triples=10, 
        corpus=corpus)),
    ('umap_embedder', EmbeddingUmapTransformer(text_column="body"))
],
    verbose=True
)


In [26]:
test_df = df.sample(100)

In [27]:
pipeline.fit(test_df)

[Pipeline] ..... (step 1 of 4) Processing topic_encoder, total=   5.8s
[Pipeline]  (step 2 of 4) Processing kw_count_vectorizer, total=  11.1s


100%|██████████| 100/100 [00:24<00:00,  4.02it/s]


[Pipeline] . (step 3 of 4) Processing triplet_extractor, total=  50.2s
[Pipeline] ..... (step 4 of 4) Processing umap_embedder, total=  32.3s


Pipeline(steps=[('topic_encoder', TopicEncoder(text_col='body')),
                ('kw_count_vectorizer',
                 KeywordVectorizer(max_features=50, text_col_name='body')),
                ('triplet_extractor',
                 TripleExtractor(corpus=['I applied for a job and after two '
                                         'interviews I was given for and '
                                         'agreed to an offer letter, had a '
                                         'start date.  The only remaining '
                                         'variable was a drug test, which I '
                                         'prepared myself for by b...
                                         'no reason. You’d think something '
                                         'representing something for LEGAL '
                                         'ADVICE would help someone looking '
                                         'for LEGAL ADVICE but apparently '
                                         'not \n'
                                         '\n'
                                         'I want to know this answer I don’t '
                                         'see what’s wrong with that. I '
                                         'GENUINELY don’t', ...],
                                 n_top_triples=10,
                                 risk_keywords=['liability', 'insurance',
                                                'should_do'],
                                 text_column='body')),
                ('umap_embedder',
                 EmbeddingUmapTransformer(text_column='body'))],
         verbose=True)

In [28]:
test_res = pipeline.transform(test_df)

100%|██████████| 100/100 [00:24<00:00,  4.04it/s]


In [29]:
test_res.head(3)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,State,kmeans_label,topic_title,multiple_topics,topic_0,topic_1,topic_2,count_vec_0,count_vec_1,count_vec_2,count_vec_3,count_vec_4,count_vec_5,count_vec_6,count_vec_7,count_vec_8,count_vec_9,count_vec_10,count_vec_11,count_vec_12,count_vec_13,count_vec_14,count_vec_15,count_vec_16,count_vec_17,count_vec_18,count_vec_19,count_vec_20,count_vec_21,count_vec_22,count_vec_23,count_vec_24,count_vec_25,count_vec_26,count_vec_27,count_vec_28,count_vec_29,count_vec_30,count_vec_31,count_vec_32,count_vec_33,count_vec_34,count_vec_35,count_vec_36,count_vec_37,count_vec_38,count_vec_39,count_vec_40,count_vec_41,count_vec_42,count_vec_43,count_vec_44,count_vec_45,count_vec_46,count_vec_47,count_vec_48,count_vec_49,top_triples_i_should_do_what,top_triples_retailer_canceled_order,top_triples_someone_does_not_plan_to_follow_through_with_the_contract,top_triples_someone_will_file_claims_case,top_triples_lawyer_had_chance,top_triples_that_did_consultation,top_triples_i_sent_retainer_agreement,top_triples_realities_do_not_reflect_commitments,top_triples_that_could_change_lives,top_triples_boss_pulled_girlfriend,liability,insurance,should_do,umap_0,umap_1,umap_2,umap_3,umap_4
0,524,1589605242,https://www.reddit.com/r/legaladvice/comments/...,gkoyov,Signed a 2 month sublease with someone in earl...,Sublease Situation,housing,7,"[-0.017685881931148586, -0.007538027247924855,...",154,"""Legal implications of breaking a sublease agr...",KS,0,Legal Topics in Rental Properties,[0],1,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.803133,0.297535,0.356883,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.372940,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0,0,1,1,0,0,0,0,0,0,0,0,0,12.023789,7.921834,2.969897,8.038216,4.074813
1,6114,1423295631,https://www.reddit.com/r/legaladvice/comments/...,2v2pbi,"Hey guys, long time (non-lawyer) lurker of the...","[US-TN] Just retained an attorney, what do I n...",family,6,"[-0.0018092953279340005, 0.021173241331556625,...",986,"""Key considerations for hiring a family law at...",ID,7,Child Custody and Related Issues,"[0, 2]",1,0,1,0.000000,0.0,0.316155,0.427604,0.0,0.104528,0.182365,0.115107,0.100197,0.0,0.0,0.0,0.418111,0.0,0.000000,0.000000,0.000000,0.511492,0.102298,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.192644,0.0,0.0,0.0,0.106901,0.00000,0.0,0.0,0.0,0.0,0.243622,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.140596,0.251371,0.000000,0.0,0.0,0.109438,0,0,0,0,1,1,1,1,0,0,0,0,0,9.225992,7.186953,3.447510,9.862123,4.102845
2,564,1590688682,https://www.reddit.com/r/legaladvice/comments/...,gsb52f,My boyfriend has a 3 year old daughter current...,Moving A Child Out of State (Oklahoma),family,6,"[0.0077508327692643215, 0.016073830068795918, ...",227,"""Seeking advice on relocating with a child ami...",MT,7,Child Custody and Related Issues,"[0, 2]",1,0,1,0.268185,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.290271,0.000000,0.280948,0.000000,0.000000,0.0,0.0,0.0,0.336988,0.0,0.0,0.0,0.317301,0.0,0.0,0.0,0.000000,0.37918,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.414029,0.0,0.0,0.0,0.0,0.000000,0.000000,0.485231,0.0,0.0,0.000000,0,0,0,0,0,0,0,0,1,0,0,0,0,9.366739,7.134408,3.828840,9.549561,4.231846


In [35]:
df_test = test_res.iloc[:, 14:]


In [36]:
test_res['target'] = np.where(test_res['text_label']=="employment", 1, 0)

In [31]:
from feature_engine.selection import ProbeFeatureSelection
from sklearn.ensemble import RandomForestClassifier

In [32]:
sel = ProbeFeatureSelection(
    estimator=RandomForestClassifier(),
    variables=None,
    scoring="precision",
    n_probes=1,
    distribution="normal",
    cv=5,
    random_state=150,
    confirm_variables=False
)

In [37]:
sel.fit(df_test, test_res['target'])

ProbeFeatureSelection(estimator=RandomForestClassifier(), random_state=150,
                      scoring='precision')

In [39]:
sel.feature_importances_.head(10)

topic_0        0.008615
topic_1        0.011466
topic_2        0.005186
count_vec_0    0.014294
count_vec_1    0.004861
count_vec_2    0.013218
count_vec_3    0.016269
count_vec_4    0.007265
count_vec_5    0.014331
count_vec_6    0.007042
dtype: float64

In [40]:
sel.features_to_drop_

['topic_0',
 'topic_1',
 'topic_2',
 'count_vec_0',
 'count_vec_1',
 'count_vec_2',
 'count_vec_3',
 'count_vec_4',
 'count_vec_5',
 'count_vec_6',
 'count_vec_7',
 'count_vec_9',
 'count_vec_10',
 'count_vec_11',
 'count_vec_12',
 'count_vec_13',
 'count_vec_14',
 'count_vec_15',
 'count_vec_16',
 'count_vec_17',
 'count_vec_18',
 'count_vec_19',
 'count_vec_20',
 'count_vec_21',
 'count_vec_22',
 'count_vec_23',
 'count_vec_24',
 'count_vec_25',
 'count_vec_26',
 'count_vec_27',
 'count_vec_28',
 'count_vec_29',
 'count_vec_30',
 'count_vec_31',
 'count_vec_32',
 'count_vec_33',
 'count_vec_34',
 'count_vec_35',
 'count_vec_36',
 'count_vec_37',
 'count_vec_38',
 'count_vec_39',
 'count_vec_40',
 'count_vec_41',
 'count_vec_42',
 'count_vec_43',
 'count_vec_44',
 'count_vec_45',
 'count_vec_46',
 'count_vec_47',
 'count_vec_48',
 'count_vec_49',
 'top_triples_i_should_do_what',
 'top_triples_retailer_canceled_order',
 'top_triples_someone_does_not_plan_to_follow_through_with_the_cont